# Import paths from environment variables

In [1]:
from dotenv import load_dotenv
import os
import geemap
import ee
import time
import pandas as pd

# Load the environment variables from the .env file
load_dotenv()

# os.environ.clear()

# Retrieve paths from environment variables file
path_to_folder = os.getenv('PATH_TO_FOLDER')

path_in = os.path.join(path_to_folder, os.getenv('NDVI_INPUT_OTHER_SALARS'))
path_out = os.path.join(path_to_folder, os.getenv('NDVI_RAW_OUTPUT_OTHERS_SALARS'))
path_out_processed = os.path.join(path_to_folder, os.getenv('NDVI_PROCESSED_OUTPUT_OTHER_SALARS'))

if path_in is None and path_out is None and path_out_processed is None:
    print("Paths not found in .env file")
else:
    print("Input path retrieved:", path_in)
    print("Output path retrieved:", path_out)
    print("Output path processed retrieved:", path_out_processed)

local_cell_coordinate_file = path_in + '/other_salar_coordinates.xlsx'
print("Local coordinates file path:", local_cell_coordinate_file)

out_landsat_annual = path_out + '/other_salar_ndvi_landsat_annual.csv'
print("Intermediate output file path:", out_landsat_annual)


Input path retrieved: /Users/vaiostriantafyllou/Library/CloudStorage/Box-Box/Projects/Chile Lithium/chile_lithium/data/raw_data/ndvi/other_salars
Output path retrieved: /Users/vaiostriantafyllou/Library/CloudStorage/Box-Box/Projects/Chile Lithium/chile_lithium/data/raw_data/ndvi/other_salars/earth_engine_output
Output path processed retrieved: /Users/vaiostriantafyllou/Library/CloudStorage/Box-Box/Projects/Chile Lithium/chile_lithium/data/processed_data/ndvi/other_salars
Local coordinates file path: /Users/vaiostriantafyllou/Library/CloudStorage/Box-Box/Projects/Chile Lithium/chile_lithium/data/raw_data/ndvi/other_salars/other_salar_coordinates.xlsx
Intermediate output file path: /Users/vaiostriantafyllou/Library/CloudStorage/Box-Box/Projects/Chile Lithium/chile_lithium/data/raw_data/ndvi/other_salars/earth_engine_output/other_salar_ndvi_landsat_annual.csv


# Set-up Google Earth Engine

In [12]:
geemap.update_package()
ee.Initialize()

Unzipping geemap-master.zip ...
Data downloaded to: /Users/vaiostriantafyllou/Downloads/geemap-master
Processing /Users/vaiostriantafyllou/Downloads/geemap-master
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for geemap: filename=geemap-0.29.6-py2.py3-none-any.whl size=2191772 sha256=587776112fcda3e2cd86f29e00fc962d66c2b6f174533ba9abdd12338e5adb32
  Stored in directory: /Users/vaiostriantafyllou/Library/Caches/pip/wheels/b2/91/55/1eaa1d48fa3ceca8723ef43560d2fffaa6d213676f9cf5f52b
Successfully built geemap
  Attempting uninstall: geemap
    Found existing installation: geemap 0.29.6
    Uninstalling geemap-0.29.6:
      Successfully uninstalled geemap-0.29.6


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.



Please comment out 'geemap.update_package()' and restart the kernel to take effect:
Jupyter menu -> Kernel -> Restart & Clear Output


# Parameters

In [2]:
# Load the Excel file
df = pd.read_excel(local_cell_coordinate_file, 'summary')

# Convert the 'coordinates_center' column to a list
origin_list = df['coordinates_center'].dropna().tolist()

# Surrounding distance for data collection
surrounding_distance = 200000

# Date range for data collection
start_date = '2013-01-01'
end_date = '2021-01-01'

# Degrees of smallest gridsize
grid_deg = 0.0027
grid_size = 300

# Functions

In [20]:
# Function for calculating distance using coordinates

def calc_distance(dataf):
    measure_coords = dataf["coordinates"]
    origin_coords = dataf["origin"]
    d = distance.distance(measure_coords, origin_coords).km
    return(d)

# Extract data from Google Earth Engine

In [48]:
# Define a function to process each row
def process_row(row):
    result = pd.DataFrame()  # Initialize an empty DataFrame to store results

    j = 1
    while f'lat{j+1}' in row.index and 'long1' and 'long2' in row.index:
        lat1_col = f'lat{j}'
        lat2_col = f'lat{j+1}'
        long1_col = 'long1'
        long2_col = 'long2'
        
        data = ee.Geometry.BBox(row[long1_col], row[lat1_col], row[long2_col], row[lat2_col])
        grid_deg = 0.0027  # Set your desired grid size in degrees
        grid_size = 300  # Set your desired pixel scale in meters

        fishnet = geemap.fishnet(data, h_interval=grid_deg, v_interval=grid_deg, delta=1)
        geemap.zonal_statistics(annual_imagery, fishnet, out_landsat_annual, statistics_type='MEAN', scale=grid_size)
        aux = pd.read_csv(out_landsat_annual)
        result = pd.concat([result, aux])

        j += 1
        time.sleep(2)

    return result


for index, origin in enumerate(origin_list):
    
    i = index+1

    if index > 0:
        continue

    parts = origin.split(',-')
    latitude, longitude = [float(coord.replace(',', '.')) for coord in parts]


    # data extraction
    selected_point = ee.Geometry.Point([longitude, latitude])
    buffer = selected_point.buffer(surrounding_distance)

    landsat_collection = ee.ImageCollection("LANDSAT/LC08/C01/T1_ANNUAL_NDVI").filterDate(start_date, end_date)\
        .filterBounds(buffer)

    annual_imagery = landsat_collection.toBands()

    local_output = os.path.join(path_out, f'other_salars_{i}.csv')

    cell_coordinates = pd.read_excel(local_cell_coordinate_file, sheet_name=index)

    # Apply the process_row function to each row of the DataFrame
    result_list = cell_coordinates.apply(process_row, axis=1)

    # Concatenate the individual results into a single DataFrame
    final_result = pd.concat(result_list.tolist(), ignore_index=True)

    # Save the final result DataFrame to a CSV file
    final_result.to_csv(local_output)

    print(f"Done with {i}")
    time.sleep(2)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /Users/vaiostriantafyllou/Desktop/chile_lithium/data/raw_data/ndvi/other_salars/earth_engine_output/other_salar_ndvi_landsat_annual.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /Users/vaiostriantafyllou/Desktop/chile_lithium/data/raw_data/ndvi/other_salars/earth_engine_output/other_salar_ndvi_landsat_annual.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /Users/vaiostriantafyllou/Desktop/chile_lithium/data/raw_data/ndvi/other_salars/earth_engine_output/other_salar_ndvi_landsat_annual.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /Users/vaiostriantafyllou/Desktop/chile_lithium/data/raw_data/ndvi/other_salars/earth_engine_output/other_salar_ndvi_landsat_annual.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /Users/vaiostriantafyllou/Desktop/chile_lithium/data/raw_data

In [ ]:



for i in range(1, 52):

    local_output = os.path.join(path_out, f'other_salars_{i}.csv')
    cell_coordinates = pd.read_excel(local_cell_coordinate_file, sheet_name=suffix)

    # Apply the process_row function to each row of the DataFrame
    result_list = cell_coordinates.apply(process_row, axis=1)

    # Concatenate the individual results into a single DataFrame
    final_result = pd.concat(result_list.tolist(), ignore_index=True)

    # Save the final result DataFrame to a CSV file
    final_result.to_csv(local_output)

    print(f"Done with {i}")
    time.sleep(2)

## Calculate distances from the center of the Salar

In [129]:
# List of file names
file_names = [f"local_{i}" for i in range(1, 52)]

# Iterate over each file, process it, and save the processed version
for idx, file_name in enumerate(file_names, start=1):
    # Load each file
    df = pd.read_csv(f"{path_out}/{file_name}.csv")

    # Drop unnecessary columns
    df = df.drop(columns=['Unnamed: 0', 'system:index'])

    # Calculate longitude, latitude, and coordinates
    df['longitude'] = (df['east'] + df['west']) / 2
    df['latitude'] = (df['north'] + df['south']) / 2
    df['coordinates'] = df['latitude'].astype(str) + ',' + df['longitude'].astype(str)

    # Assign origin and calculate distance
    df["origin"] = origin
    df['distance'] = df.apply(calc_distance, axis=1)

    # Save the processed file
    df.to_csv(f"{path_out_processed}/{file_name}_with_distances.csv", index=False)
